In [1]:
import os
import openai
import pandas as pd
from openai import OpenAI, AzureOpenAI
# from openai.embeddings_utils import get_embedding
from dotenv import load_dotenv
load_dotenv()

True

- For specific conda, to create kernel. `python -m ipykernel install --user --name=qa-pgvector`
- https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084

In [2]:
CSV_FILE = "/Users/spare/Documents/data/articles.csv"
df = pd.read_csv(CSV_FILE)
df.head(5)

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,garment_group_name,detail_desc
0,695255001,Siv t-shirt,T-shirt,Garment Upper body,All over pattern,Dark Blue,Jersey Fancy,Short-sleeved top in soft viscose jersey with ...
1,821115007,RICHIE SKIRT,Skirt,Garment Lower body,Check,Pink,Skirts,"Short, pleated skirt in woven fabric with a hi..."
2,553238008,THORN LS TEE,Sweater,Garment Upper body,Solid,White,Jersey Basic,"Wide, long-sleeved top in soft cotton jersey w..."
3,627147001,Bling Me Up Push,Bikini top,Swimwear,Lace,Dark Red,Swimwear,"Fully lined bikini top with hole-patterned, un..."
4,794520001,Plus seam at back 1p tights,Underwear Tights,Socks & Tights,Solid,Black,Socks and Tights,Tights with a seam down the back of the legs. ...


In [3]:
API_BASE = os.getenv('API_BASE')
API_KEY = os.getenv('API_KEY')
API_VERSION = os.getenv('API_VERSION')
CHAT_DEPLOYMENT_NAME = os.getenv('CHAT_DEPLOYMENT_NAME')
EMBEDDING_DEPLOYMENT_NAME = os.getenv('EMBEDDING_DEPLOYMENT_NAME')
# print(API_BASE, API_KEY, API_VERSION, CHAT_DEPLOYMENT_NAME, EMBEDDING_DEPLOYMENT_NAME)

In [4]:
client = AzureOpenAI(
    azure_endpoint=API_BASE,
    api_version=API_VERSION,
    api_key=API_KEY
    # deployment_name=CHAT_DEPLOYMENT_NAME,
    # model_name=EMBEDDING_DEPLOYMENT_NAME
)
completion = client.chat.completions.create(
    model=CHAT_DEPLOYMENT_NAME,
    messages=[
        {
            "role": "user",
            "content": "How do I output all files in a directory using Python?",
        },
    ],
)
print(completion.model_dump_json(indent=2))

{
  "id": "chatcmpl-8IRC6VlIpXVmvAfNXzdVgrwOPonpK",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "You can use the `os` module in Python to output all files in a directory. Here's an example:\n\n```python\nimport os\n\n# Specify the directory path\ndirectory = \"path/to/directory\"\n\n# Iterate over all files in the directory\nfor filename in os.listdir(directory):\n    # Check if the current item is a file\n    if os.path.isfile(os.path.join(directory, filename)):\n        # Print the filename\n        print(filename)\n```\n\nReplace `\"path/to/directory\"` with the actual path of the directory you want to list the files from. This code will print the name of each file in the specified directory.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "sel

In [6]:
client.embeddings.create(input=['I am Leo'], model=EMBEDDING_DEPLOYMENT_NAME)

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.006654671858996153, 0.010415453463792801, -0.020269976928830147, -0.027587566524744034, -0.05112113803625107, 0.021404586732387543, -0.031310103833675385, 0.008439449593424797, -0.005864270031452179, -0.019632557407021523, 0.00509936548769474, 0.0191353689879179, 0.006680168677121401, 0.005574243608862162, -0.004302589688450098, -0.010460073128342628, 0.029831288382411003, -0.008439449593424797, 0.03577204793691635, 0.01569329760968685, 0.010083994828164577, -0.011945263482630253, 0.007266595494002104, 0.001716255210340023, 0.014596933498978615, -0.013462324626743793, 0.020308222621679306, -0.01746532693505287, 0.010396331548690796, -0.03551708161830902, 0.017197608947753906, -0.007935887202620506, 0.007107240613549948, -0.033222366124391556, -0.00616704486310482, -0.02840346470475197, 0.004344021901488304, -0.012340464629232883, 0.006125612650066614, 0.008496817201375961, 0.011728540062904358, 0.005370269063860178, -0.0033655811566